## 네이버 지식IN - 추석 선물

In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib.parse import quote
from tqdm import tqdm

- 지식인 추석 선물 검색결과 화면

In [3]:
url = f'https://kin.naver.com/search/list.naver?query={quote("추석 선물")}&page=1'
driver = webdriver.Chrome()
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [4]:
lis = soup.select('.basic1 > li')
len(lis)

10

- 답변 갯수가 5개보다 큰 항목을 선택

In [6]:
li = lis[7]
sub_url = li.select_one('a._searchListTitleAnchor')['href']
sub_url

'https://kin.naver.com/qna/detail.naver?d1id=6&dirId=6130103&docId=454639453&qb=7LaU7ISdIOyEoOusvA==&enc=utf8&section=kin&rank=8&search_sort=0&spq=0'

In [7]:
driver.get(sub_url)
time.sleep(2)

In [9]:
# 더 보기 클릭 횟수 구하기
answer_num = driver.find_element(By.CSS_SELECTOR, '._answerCount.num').text
click_count = (int(answer_num) - 1) // 5
answer_num, click_count

('8', 1)

In [10]:
# 더 보기 횟수 만큼 클릭하기
for _ in range(click_count):
    driver.find_element(By.ID, 'nextPageButton').click()
    time.sleep(2)

In [11]:
# 펼쳐진 화면을 BeautifulSoup으로 파싱
sub_soup = BeautifulSoup(driver.page_source, 'html.parser')
divs = sub_soup.select('.answer-content__list._answerList > div')
len(divs)

8

In [12]:
# 답변 목록에서 개별 답변을 선택하고 글을 추출하기
div = divs[0]
ps = div.select('.se-text-paragraph.se-text-paragraph-align-')
len(ps)

45

In [18]:
answer = ''
for p in ps:
    text = p.get_text().strip()
    if text == '\u200b' or text == '':
        continue
    answer += text + '\n'
print(answer)

많은 인원수 준비 하실려면 부담이 크시겠어요
연령대도 다양하시고
남녀노소 가리지 않고 해드릴수 있는 선물을 추천 해드릴려고 합니다
곧 풍성한 추석이 몇일 남지 않았어요
부모님추석선물 어떤거 할지 고민이실까요 ?
시부모님 친정부모님 우리집 가장인 신랑 그리고아내들 까지 모두에게 꼭필요한
30~40대부터 70~80대 어르신들까지 건강 지킴이 필수 요건 _ <면역기능 강화>
면역력이 튼튼해야지만 어떠한 병균에도 이겨낼수 있습니다
광동 맑은 365 면역보감은
세계 최고 품질 "호주산 프로폴리스"
면역기능의 필수영양소 "아연"
빠른 흡수를 위해  [액상형]으로
식약처 인정한 엄선된 재료들만 골라 한팩안에 담았습니다
※이런분들에게추천드립니다 ※
만성피로로 힘드신분, 급격하게 체력이 저하되시는분
무기력함을 느끼시는분 기력증,원기충전이 필요하신분
허약체질, 잔병치레로 관리 필요하신분, 꾸준한 건강관리를 원하시는분
지금 한가위 맞이하여 특별 이벤트를 진행하고 있습니다
무료상담 받아보세요
▼▼▼▼▼▼▼▼▼▼▼
http://vadmymll.adpt.kr
답변채택부탁드립니다 ^^
#부모님추석선물 #부모님건강선물 #면역기능강화 #면역보감 #광동맑은365면역보감 #프로폴리스



In [19]:
# 답변 목록에 있는 모든 답변을 선택해서 글을 추출하기
answer_list = []
for div in divs:
    ps = div.select('.se-text-paragraph.se-text-paragraph-align-')
    answer = ''
    for p in ps:
        text = p.get_text().strip()
        if text == '\u200b' or text == '':
            continue
        answer += text + '\n'
    answer_list.append(answer)

In [20]:
len(answer_list)

8

In [21]:
driver.back()

- 한 화면에 있는 10개의 글에서 모든 답변을 추출하기

In [22]:
answer_list = []
for li in lis:
    sub_url = li.select_one('a._searchListTitleAnchor')['href']
    driver.get(sub_url)
    time.sleep(2)

    answer_num = driver.find_element(By.CSS_SELECTOR, '._answerCount.num').text
    click_count = (int(answer_num) - 1) // 5 
    print(click_count, end=' ')
    for _ in range(click_count):
        driver.find_element(By.ID, 'nextPageButton').click()
        time.sleep(2)

    sub_soup = BeautifulSoup(driver.page_source, 'html.parser')
    divs = sub_soup.select('.answer-content__list._answerList > div')
    for div in divs:
        ps = div.select('.se-text-paragraph.se-text-paragraph-align-')
        answer = ''
        for p in ps:
            text = p.get_text().strip()
            if text == '\u200b' or text == '':
                continue
            answer += text + '\n'
        answer_list.append(answer)

    driver.back()
    time.sleep(2)

0 0 0 0 0 0 0 1 1 3 

In [23]:
len(answer_list)

47

- 1 ~ 10 페이지에 대한 답변 가져오기

In [25]:
answer_list = []
driver = webdriver.Chrome()
for page in range(1, 11):
    url = f'https://kin.naver.com/search/list.naver?query={quote("추석 선물")}&page={page}'
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('.basic1 > li')

    for li in lis:
        sub_url = li.select_one('a._searchListTitleAnchor')['href']
        driver.get(sub_url)
        time.sleep(2)

        answer_num = driver.find_element(By.CSS_SELECTOR, '._answerCount.num').text
        click_count = (int(answer_num) - 1) // 5 
        print(click_count, end=' ')
        for _ in range(click_count):
            driver.find_element(By.ID, 'nextPageButton').click()
            time.sleep(2)

        sub_soup = BeautifulSoup(driver.page_source, 'html.parser')
        divs = sub_soup.select('.answer-content__list._answerList > div')
        for div in divs:
            ps = div.select('.se-text-paragraph.se-text-paragraph-align-')
            answer = ''
            for p in ps:
                text = p.get_text().strip()
                if text == '\u200b' or text == '':
                    continue
                answer += text + '\n'
            answer_list.append(answer)

        driver.back()
        time.sleep(2)
    print()

driver.close()

  0%|          | 0/10 [00:00<?, ?it/s]

0 0 0 0 0 0 0 1 1 3 

 10%|█         | 1/10 [01:05<09:47, 65.30s/it]


1 5 0 1 

 10%|█         | 1/10 [01:41<15:09, 101.03s/it]


WebDriverException: Message: no such execution context
  (Session info: chrome=117.0.5938.89)
Stacktrace:
	GetHandleVerifier [0x00007FF7137D78A2+54818]
	(No symbol) [0x00007FF713746AD2]
	(No symbol) [0x00007FF7135FD8ED]
	(No symbol) [0x00007FF7135EC5B9]
	(No symbol) [0x00007FF7135EA961]
	(No symbol) [0x00007FF7135EB2EE]
	(No symbol) [0x00007FF7135F7D4D]
	(No symbol) [0x00007FF7136084D1]
	(No symbol) [0x00007FF71360C80A]
	(No symbol) [0x00007FF7135EB937]
	(No symbol) [0x00007FF713608163]
	(No symbol) [0x00007FF713677609]
	(No symbol) [0x00007FF71365E883]
	(No symbol) [0x00007FF713633691]
	(No symbol) [0x00007FF7136348D4]
	GetHandleVerifier [0x00007FF713B3B9A2+3610402]
	GetHandleVerifier [0x00007FF713B91870+3962352]
	GetHandleVerifier [0x00007FF713B89D5F+3930847]
	GetHandleVerifier [0x00007FF713873656+693206]
	(No symbol) [0x00007FF713751638]
	(No symbol) [0x00007FF71374D944]
	(No symbol) [0x00007FF71374DA72]
	(No symbol) [0x00007FF71373E123]
	BaseThreadInitThunk [0x00007FFBE04E7344+20]


In [26]:
len(answer_list)

90

In [27]:
answer_list[-1]

''

In [28]:
answer_list[-5:]

['추석이 다가오면 언제나 선물 고민이 시작되죠?\n어떤 선물을 하면 좋을지, 가격은 얼마가 적당할지, 받는 분이 좋아할지 고민이 많으실 텐데요.\n저도 매번 이런 고민을 하다 추석 선물 여러 종류를 추천하는 영상 하나를 만들어 봤어요\n대기업에서 판매하는 기름, 햄, 참치, 비누 세트 등은 뺐구요,\n좀 더 새롭고,\n어떤 선물을 할지 고민해본 흔적이 있는 선물들만 넣어놨어요\n한 번 보시고 참고하셔서 따뜻한 마음 잘 전하시길 바래요.\n',
 '안녕하세요~ 소뱅이네 입니다.\n추석이 지나면 날씨도 추워지는데 포근한 이불은 어떨까요?\n매일 사용하는 제품이라 실용적이고 기억에 오래 남을 것 같습니다.\n수년간 오프라인 매장 운영 경험을 바탕으로 온라인 판매를 시작했습니다.\n국내생산 고품질의 합리적인 가격의 원하시는 이불을 소개해드려요.\n아래의 제품 이외에도 많은 제품이 있으니 방문 한번 부탁드립니다.\n',
 '아내되실분과 먼저는 꼭 상의를 하시고.. ㅎㅎ\n아무래도 선물은 고기나 과일이 가격적으로도 서로 부담안되고 기분좋게 할 수 있는 선물인 것 같아요\n최근에 장인 장모님 선물 구매하고 만족했던 사이트 링크 남겨드릴테니 도움되시길 바랍니다 ㅎㅎ\n근데 당근어플이 설치되어 있어야 들어가실 수 있어요!\n도움되셨다면 채택도 부탁드릴게요 ㅎㅎ\n',
 '',
 '']